# Projeto Final

O objetivo deste projeto é criar as bases de um sistema capaz de auxiliar deficientes visuais na identificação de objetos no mundo à sua volta.

Primeiramente é necessário garantir que todas as dependências estão instaladas.

In [53]:
!pip install -r requirements.txt

Estes são todos os imports necessários para rodar este Notebook.

In [54]:
import torch
import pandas as pd
from gtts import gTTS

Agora vamos carregar o modelo pré-treinado do hub do Torch. Esse modelo é baseado na YOLOv5 e está pré-treinada com a base COCO Dataset. Caso seja a primeira execução, a API vai realizar o download automático do modelo. Mas caso o modelo já tiver sido baixado, a API do Torch consegue reaproveitar o arquivo, ficando mais rápida a carga.

Esse dataset de imagens possui 80 categorias de objetos diversos. Futuramente é possível treinar o modelo com outros datasets, como o do OpenImages do Google, que possui 600 categorias por exemplo.

O endereço do projeto YOLOv5 é: https://github.com/ultralytics/yolov5
O endereço do projeto COCO Dataset é: https://cocodataset.org/

In [55]:
# Load Model (yolov5s, yolov5x6)
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

Using cache found in /Users/diego/.cache/torch/hub/ultralytics_yolov5_master
INFO:yolov5:requirements: YOLOv5 requirement "seaborn>=0.11.0" not found, attempting AutoUpdate...
requirements: YOLOv5 requirement "seaborn>=0.11.0" not found, attempting AutoUpdate...
INFO:yolov5:Requirement already satisfied: seaborn>=0.11.0 in ./.venv/lib/python3.10/site-packages (0.12.1)


INFO:yolov5:requirements: 1 package updated per /Users/diego/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

requirements: 1 package updated per /Users/diego/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

INFO:yolov5:YOLOv5 🚀 2022-10-23 Python-3.10.7 torch-1.12.1 CPU

YOLOv5 🚀 2022-10-23 Python-3.10.7 torch-1.12.1 CPU

INFO:yolov5:Fusing layers... 
Fusing layers... 
INFO:yolov5:YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Y

Esta função recebe uma imagem, que é então enviada para o modelo realizar a detecção dos objetos contidos na mesma. A função retorna um dataframe do Pandas com as informações de todos os objetos identificados.

In [56]:
def infer_img(img):
    '''
    Esta função recebe uma imagem e retorna um Pandas dataframe com os objetos identificados.
    '''
    results = model(img)

    df = pd.DataFrame(results.pandas().xyxy[0])
    return df

Esta função recebe o dataframe gerado e é responsável por "contar" os objetos dentro da imagem. Ela retorna um dicionário onde o 'key' é o nome do objeto e o 'value' é o número de ocorrências dentro da imagem.

In [57]:
def get_result(df):
    result = {}
    def count_names(name):
        i = 0
        try:
            i = result[name]
        except:
            pass
        i += 1
        result[name] = i

    df['name'].map(count_names)
    return result

Estas duas funções são responsáveis por gerar um texto reprensentando os objetos encontrados (ou não) na imagem. A função 'get_text' retorna uma string com a descrição dos objetos da imagem.

In [58]:
def get_plural(word, count):
    if count > 1:
        return word + 's'
    else:
        return word

def get_text(result):
    text = ''
    if len(result) == 0:
        text = 'I wasn\'t able to identify anything in front of you.'
    else:
        text = 'There are '
        i = 0
        for name in result:
            i += 1
            count = result[name]
            text += str(count) + ' ' + get_plural(name, count)
            if i == len(result) - 1:
                text += ' and '
            elif i == len(result):
                text += ' '
            else:
                text += ', '
        text += 'in front of you'
    return text

Esta função recebe o texto gerado o transforma em áudio usando a API do Google Text-to-Speech. Por fim, ela executa o áudio gerado usando a biblioteca 'playsound'.

In [48]:
def play(text):
    if not text:
        return
    
    lang = 'en'

    gtts_obj = gTTS(text=text, lang=lang)
    gtts_obj.save('audio.wav')

    print(text)
    from playsound import playsound
    playsound('audio.wav')

Esta função é responsável por executar o código externo que tira uma foto usando a câmera padrão do dispositivo. Aqui é usado OpenCV para acessar a câmera, tirar a foto e salvá-la temporariamente.

In [59]:
def take_photo():
    !python camera.py

Este é o código que poderá ser executado sempre que for necessário identificar os objetos. Posteriormente a idéia é fazer com que o sistema execute esse código atravez de um comando de voz, tornando a iteração manual quase inexistente e facilitando a acessibilidade do usuário final.

In [60]:

take_photo()
df = infer_img('image.jpg')
result = get_result(df)
text = get_text(result)
play(text)

There are 1 person in front of you
